Install requirements.txt + google chrome

Imports

In [ ]:
from CloudflareBypasser import CloudflareBypasser
import time
from DrissionPage import SessionPage
from DrissionPage import ChromiumPage
from random import randint
import json

Retrieve server invites

In [ ]:
number-of-scrolls = 20

def save_page_html(driver, filename): # Save the page HTML to a file
    html_content = driver.html
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content) 

def detect_duplicates(invite): # Check whether the invite is already in the file
    with open('server_invites.txt') as f:
        existing_invites = f.readlines()
        existing_invites = [invite.strip() for invite in existing_invites]

    if invite in existing_invites:
        return
    else:
        with open('server_invites.txt', 'a') as f:
            f.write(invite + '\n')
        return

driver = ChromiumPage()
# driver = webdriver.Chrome() - doesn't work with Cloudflare
with open('topics.txt') as f: # You can enter your topics of interest in the topics.txt file
    topics = f.readlines()
    topics = [topic.strip() for topic in topics]

for topic in topics: # Loop through the topics
    driver.get(f'https://disboard.org/servers/tag/{topic}?sort=-member_count')

    cf_bypasser = CloudflareBypasser(driver) # Bypass Cloudflare protection
    cf_bypasser.bypass()

    time.sleep(2)  # Adjust the sleep time if necessary

    try:
        # Look for a button with text containing "Accept" or "Agree"
        accept_button = driver.ele('tag:button@text:Accept all & visit the site') # Turns out you dont need to get rid of the dialogue
        if not accept_button:
            accept_button = driver.ele('tag:button@text:Agree')
        
        if accept_button:
            accept_button.click()
            time.sleep(1)  # Wait for the dialogue to disappear
        else:
            print("Could not find cookie accept button")
    except Exception as e:
        print(f"An error occurred while handling cookie dialogue: {e}")

    # Save the page HTML to a file
    # save_page_html(driver, 'disboard.html')

    for i in range(number-of-scrolls):  # Adjust the number of scrolls as needed
        driver.scroll.down(randint(1246, 2539))  # Scroll down by a random amount not to get detected
        time.sleep(3)  # Wait for content to load (It's a single countinuous page)

    js_script = '''
        var links = document.querySelectorAll('a[href*="/server/join/"]');
        var invites = [];
        links.forEach(function(link) {
            invites.push(link.href);
        });
        return invites;
    '''

    discord_invites = driver.run_js(js_script) # detect invite links

    # Print the found Discord invite links
    for invite in discord_invites:
        detect_duplicates(invite)
    print("number of invites: ", len(discord_invites))


# driver.set.cookies({'name': 'cookie_name', 'value': 'cookie_value'})

# accept_button = driver.find_element(By.XPATH, "//button[text()='Accept all & visit the site']")
# accept_button.click()

Get discord invites from disboard links

In [ ]:
# Function to get the actual Discord invite from Disboard
def get_discord_invite(disboard_url):
    # Start Chrome using Selenium WebDriver
    driver = ChromiumPage()
    
    try:
        # Open the Disboard URL
        driver.get(disboard_url)

        # Wait for redirection to Discord (you can adjust the timeout)
        time.sleep(5)

        # After the redirect, get the current URL
        current_url = driver.url

        # Check if it is a valid Discord invite URL
        if "discord.com/invite" in current_url:
            # Extract the invite code
            invite_code = current_url.split('/')[-1]
            return invite_code
        else:
            return None
    except Exception as e:
        print(f"Error processing {disboard_url}: {e}")
        return None
    finally:
        pass
        # Quit the browser (optional)
        # driver.quit()

# Read Disboard invites from the file
with open("server_invites_gg.txt", "r") as file:
    disboard_links = file.readlines() # Load invites

# Open a new file to save the actual Discord invites
with open("invites_discord_format.txt", "w") as output_file:
    for disboard_url in disboard_links:
        disboard_url = disboard_url.strip()  # Remove newline and spaces
        if disboard_url:
            # Get Discord invites from the Disboard URLs
            invite_code = get_discord_invite(disboard_url)
            if invite_code:
                # Write the Discord invite code to the file
                output_file.write(f"https://discord.com/invite/{invite_code}\n")
                print(f"Processed: {disboard_url} -> https://discord.com/invite/{invite_code}")
            else:
                print(f"Failed to process: {disboard_url}")


Accepting the invites (not finished)

In [ ]:
driver = ChromiumPage()

with open('server_invites.txt') as f:
    existing_invites = f.readlines()
    existing_invites = [invite.strip() for invite in existing_invites]

def log_in_to_discord():
    driver.get('https://discord.com/login')
    time.sleep(5)
    try:
        with open('login.json') as login_file:
            login_data = json.load(login_file)
            mail_address = login_data['email']
            password = login_data['password']
        login_button = driver.ele('tag:button@text:Log In')
        email_input = driver.ele('tag:input@name:email')
        email_input.input(f'{mail_address}')
        password_input = driver.ele('tag:input@name:password')
        password_input.input(f'{password}')
        login_button.click()
        
        cookies = driver.get_cookies()  # Retrieve cookies from the current session
        with open('cookies.json', 'w') as f:
            json.dump(cookies, f)

        driver.set_cookies(cookies)  # Apply the previously saved cookies

        # Refresh the page to ensure cookies are applied correctly
        driver.refresh()

    except Exception as e:
        print(f"An error occurred while logging in: {e}")

def join_a_server(invite_link):
    driver.get(invite_link)
    time.sleep(5)
    try:
        join_button = driver.ele('tag:button@text:Join Server')
        join_button.click()
        time.sleep(5)
    except Exception as e:
        print(f"An error occurred while joining the server: {e}")

log_in_to_discord()

time.sleep(5)

# for invite in existing_invites:
join_a_server(existing_invites[1])


Scraping messages